In [9]:
import sys
sys.path.append('../api/src')


In [15]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    JSONAgentOutputParser,
    ReActSingleInputOutputParser,
)
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun, tool
from langchain.tools.render import render_text_description_and_args, format_tool_to_openai_function
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder
from langchain.schema import ChatMessage, SystemMessage

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
)

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"

In [18]:
import re
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

mp_llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    max_retries=5,
    streaming=True,
	callbacks=[StreamingStdOutCallbackHandler()],
)

llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    streaming=True,
	callbacks=[StreamingStdOutCallbackHandler()],
)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = [
    MPThermoExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElasticityExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPDielectricExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPMagnetismExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElectronicExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPSummaryExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    # arxiv,
    # wikipedia,
]

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages[0].prompt.template = re.sub(
    r"\s+", " ",
    """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    user to clarify their queries if needed. Please note that you don't have 
    direct control over MP but through multiple assistant agents to help you. 
    You need to provide complete context in the input for them to do their job.
    """).replace("\n", " ") + prompt.messages[0].prompt.template

prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm.bind(stop=["Observation"])
    # | map_reduce_chain  # TODO: Add map-reduce after LLM
    | JSONAgentOutputParser()
)


conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

agent_kwargs = {
    "handle_parsing_errors": True,
    "extra_prompt_messages": [
        MessagesPlaceholder(variable_name="chat_history"),
        ],
}

agent_executor = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10,
    memory=conversational_memory,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
)

/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [19]:
agent_executor.invoke({
    "input": "What's the bandgap of CsPbI3?"
})



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "MPElectronicExpert",
  "action_input": {
    "input": "What's the bandgap of CsPbI3?"
  }
}
```Action:
```json
{
  "action": "MPElectronicExpert",
  "action_input": {
    "input": "What's the bandgap of CsPbI3?"
  }
}
```

> Entering new AgentExecutor chain...


/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Action:
```
{
  "action": "search_materials_electronic_structure__get",
  "action_input": {
    "formula": "CsPbI3"
  }
}
```Action:
```
{
  "action": "search_materials_electronic_structure__get",
  "action_input": {
    "formula": "CsPbI3"
  }
}
```{"formula": "CsPbI3"}


Retrieving ElectronicStructureDoc documents: 100%|██████████| 3/3 [00:00<00:00, 50131.12it/s]


[{'material_id': 'mp-1120768', 'last_updated': '2023-08-12T09:16:31.142000'}, {'material_id': 'mp-1069538', 'last_updated': '2023-08-13T07:19:55.398000'}, {'material_id': 'mp-540839', 'last_updated': '2023-08-13T15:45:51.364000'}]Action:
```
{
  "action": "search_materials_electronic_structure__get",
  "action_input": {
    "formula": "CsPbI3",
    "fields": "material_id,formula_pretty,band_gap"
  }
}
```Action:
```
{
  "action": "search_materials_electronic_structure__get",
  "action_input": {
    "formula": "CsPbI3",
    "fields": "material_id,formula_pretty,band_gap"
  }
}
```{"formula": "CsPbI3", "fields": "material_id,formula_pretty,band_gap"}


Retrieving ElectronicStructureDoc documents: 100%|██████████| 3/3 [00:00<00:00, 30030.82it/s]


[{'band_gap': 1.6421000000000001, 'formula_pretty': 'CsPbI3', 'material_id': 'mp-1120768'}, {'band_gap': 1.4785000000000001, 'formula_pretty': 'CsPbI3', 'material_id': 'mp-1069538'}, {'band_gap': 2.5181, 'formula_pretty': 'CsPbI3', 'material_id': 'mp-540839'}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The band gaps for CsPbI3 are as follows: 1.6421 eV for material mp-1120768, 1.4785 eV for material mp-1069538, and 2.5181 eV for material mp-540839."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The band gaps for CsPbI3 are as follows: 1.6421 eV for material mp-1120768, 1.4785 eV for material mp-1069538, and 2.5181 eV for material mp-540839."
}
```

> Finished chain.

Observation: {'input': "What's the bandgap of CsPbI3?", 'output': 'The band gaps for CsPbI3 are as follows: 1.6421 eV for material mp-1120768, 1.4785 eV for material mp-1069538, and 2.5181 eV for material mp-540839.'}
Thought:Action:
```json
{
  "action": "Final Answer",
  "action_input

{'input': "What's the bandgap of CsPbI3?",
 'chat_history': [],
 'output': 'The band gaps for CsPbI3 are as follows: 1.6421 eV for material mp-1120768, 1.4785 eV for material mp-1069538, and 2.5181 eV for material mp-540839.'}